In [2]:
!pip install pandas openpyxl
!pip install tabulate
!pip install linearmodels


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import statsmodels.api as sm
from linearmodels.system import SUR
from statsmodels.tsa.api import VAR

In [4]:
# Load the Excel file
file_path = "Assignment2Data_2025.xlsx"

# Read a specific sheet
q1 = pd.read_excel(file_path, sheet_name="Q1", engine='openpyxl')

bonus = pd.read_excel(file_path, sheet_name="BonusQuestion", engine='openpyxl')

# Display first few rows
print(q1.head())
print(bonus.head())

   Calendar Date  Value-Weighted Return-incl. dividends (RY)  \
0       19271231                                    0.328720   
1       19281231                                    0.391397   
2       19291231                                   -0.150955   
3       19301231                                   -0.289005   
4       19311231                                   -0.443880   

   Value-Weighted Return-excl. dividends (RY)     Level  Dividend        DG  \
0                                    0.266483  1.320979  0.064917  0.170843   
1                                    0.337056  1.766222  0.071783  0.105764   
2                                   -0.182395  1.444072  0.055530 -0.226421   
3                                   -0.321455  0.979868  0.046860 -0.156130   
4                                   -0.476975  0.512495  0.032429 -0.307968   

        CPI  Real level  Real dividend    Real R   Real DG        DP        DE  
0  0.966868    1.366245       0.067142  0.358098  0.196734 

/usr/local/python/3.12.1/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# 1

In [5]:
data_1973_2024 = q1[q1['Calendar Date'] >= 19731231]

# Function to compute statistics
def compute_stats(df):
    stats = {
        "Mean Total Return": round(df["Real R"].mean(),3),
        "Std Total Return": round(df["Real R"].std(),3),
        "Mean Dividend Growth": round(df["Real DG"].mean(),3),
        "Std Dividend Growth": round(df["Real DG"].std(),3),
        "Correlation": round(df[["Real R", "Real DG"]].corr().iloc[0, 1],3)
    }
    return stats

# Compute statistics for both periods
stats_1927_2024 = compute_stats(q1)
stats_1973_2024 = compute_stats(data_1973_2024)

# Create a DataFrame to report results
report = pd.DataFrame([stats_1927_2024, stats_1973_2024], index=["1927-2024", "1973-2024"])
print(report)

           Mean Total Return  Std Total Return  Mean Dividend Growth  \
1927-2024              0.088             0.198                 0.028   
1973-2024              0.080             0.177                 0.034   

           Std Dividend Growth  Correlation  
1927-2024                0.145        0.637  
1973-2024                0.147        0.657  


In [6]:
table1 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Mean Total Return", stats_1927_2024['Mean Total Return'], stats_1973_2024['Mean Total Return']],
    ["Std Dev Total Return", stats_1927_2024['Std Total Return'], stats_1973_2024['Std Total Return']],
    ["Mean Dividend Growth", stats_1927_2024['Mean Dividend Growth'], stats_1973_2024['Mean Dividend Growth']],
    ["Std Dev Dividend Growth", stats_1927_2024['Std Dividend Growth'], stats_1973_2024['Std Dividend Growth']],
    ["Correlation (Returns & Growth)", stats_1927_2024['Correlation'], stats_1973_2024['Correlation']],
]

# Print table in Markdown format
print("Table 1. 1927-2024 and 1973-2024 Total Returns, Net Dividend Growth, and Correlations")
print(tabulate(table1, headers="firstrow", tablefmt="pretty"))

Table 1. 1927-2024 and 1973-2024 Total Returns, Net Dividend Growth, and Correlations
+--------------------------------+-----------+-----------+
|             Metric             | 1927-2024 | 1973-2024 |
+--------------------------------+-----------+-----------+
|       Mean Total Return        |   0.088   |   0.08    |
|      Std Dev Total Return      |   0.198   |   0.177   |
|      Mean Dividend Growth      |   0.028   |   0.034   |
|    Std Dev Dividend Growth     |   0.145   |   0.147   |
| Correlation (Returns & Growth) |   0.637   |   0.657   |
+--------------------------------+-----------+-----------+


# 2

In [7]:
# 1927-2024
# dividend yield
X = q1['DP'][:-1].reset_index(drop=True)

# real return
Y = q1['Real R'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression, correct for heteroskedasticity
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [8]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DP']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DP']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [9]:
# 1973-2024
# dividend yield
X = data_1973_2024['DP'][:-1].reset_index(drop=True)

# real return
Y = data_1973_2024['Real R'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [10]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DP']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DP']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [11]:
table2 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 2. Regression Results of 1927-2024 and 1973-2024 Real Returns on Dividend Yield")
print(tabulate(table2, headers="firstrow", tablefmt="pretty"))

Table 2. Regression Results of 1927-2024 and 1973-2024 Real Returns on Dividend Yield
+-----------------------------+-----------+-----------+
|           Metric            | 1927-2024 | 1973-2024 |
+-----------------------------+-----------+-----------+
|          Constant           |  -0.005   |   0.03    |
|    Constant t-statistics    |  -0.108   |   0.459   |
|       Dividend Yield        |   2.469   |   2.023   |
| Dividend Yield t-statistics |   1.89    |   1.048   |
|     Adjusted R-Squares      |   0.027   |  -0.001   |
+-----------------------------+-----------+-----------+


# 3

In [12]:
# 1927-2024
# dividend yield
X = q1['DP'][:-1].reset_index(drop=True)

# dividend growth
Y = q1['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [13]:
model_1927_2024.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Real DG   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                   0.09096
Date:                Sun, 23 Feb 2025   Prob (F-statistic):              0.764
Time:                        22:13:42   Log-Likelihood:                 50.363
No. Observations:                  97   AIC:                            -96.73
Df Residuals:                      95   BIC:                            -91.58
Df Model:                           1                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0368      0.040      0.917      0.359      -0.042       0.116
DP            -0.2964      0.983     -0.302      0.763      -2.223       1.630
==============================================================================
Omnibus:                        3.182   Durbin-Watson:                   2.351
Prob(Omnibus):                  0.204   Jarque-Bera (JB):                2.709
Skew:                           0.404   Prob(JB):                        0.258
Kurtosis:                       3.135   Cond. No.                         65.3
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [14]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DP']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DP']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [15]:
# 1973-2024
# dividend yield
X = data_1973_2024['DP'][:-1].reset_index(drop=True)

# real dividend growth
Y = data_1973_2024['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [16]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DP']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DP']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [17]:
table3 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 3. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Dividend Yield")
print(tabulate(table3, headers="firstrow", tablefmt="pretty"))

Table 3. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Dividend Yield
+-----------------------------+-----------+-----------+
|           Metric            | 1927-2024 | 1973-2024 |
+-----------------------------+-----------+-----------+
|          Constant           |   0.037   |   0.044   |
|    Constant t-statistics    |   0.917   |   0.755   |
|       Dividend Yield        |  -0.296   |  -0.241   |
| Dividend Yield t-statistics |  -0.302   |  -0.133   |
|     Adjusted R-Squares      |   -0.01   |   -0.02   |
+-----------------------------+-----------+-----------+


# 4

In [18]:
# 1927-2024
# payout ratio
X = q1['DE'][:-1].reset_index(drop=True)

# dividend growth
Y = q1['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1927_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [19]:
model_1927_2024.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Real DG   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                   0.05906
Date:                Sun, 23 Feb 2025   Prob (F-statistic):              0.809
Time:                        22:13:42   Log-Likelihood:                 50.429
No. Observations:                  97   AIC:                            -96.86
Df Residuals:                      95   BIC:                            -91.71
Df Model:                           1                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0088      0.068      0.129      0.897      -0.125       0.142
DE             0.0312      0.128      0.243      0.808      -0.220       0.283
==============================================================================
Omnibus:                        2.745   Durbin-Watson:                   2.318
Prob(Omnibus):                  0.253   Jarque-Bera (JB):                2.322
Skew:                           0.376   Prob(JB):                        0.313
Kurtosis:                       3.101   Cond. No.                         5.88
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [20]:
a_1927 = model_1927_2024.params['const']
b_1927 = model_1927_2024.params['DE']
t_stat_a_1927 = model_1927_2024.tvalues['const']
t_stat_b_1927 = model_1927_2024.tvalues['DE']
adj_r2_1927 = model_1927_2024.rsquared_adj

In [21]:
# 1973-2024
# dividend yield
X = data_1973_2024['DE'][:-1].reset_index(drop=True)

# real dividend growth
Y = data_1973_2024['Real DG'][1:].reset_index(drop=True)
    
# Add constant term for regression
X = sm.add_constant(X)
    
# Run OLS regression
model_1973_2024 = sm.OLS(Y, X).fit(cov_type='HC3')

In [22]:
a_1973 = model_1973_2024.params['const']
b_1973 = model_1973_2024.params['DE']
t_stat_a_1973 = model_1973_2024.tvalues['const']
t_stat_b_1973 = model_1973_2024.tvalues['DE']
adj_r2_1973 = model_1973_2024.rsquared_adj

In [23]:
table4 = [
    ["Metric", "1927-2024", "1973-2024"],
    ["Constant", round(a_1927,3), round(a_1973,3)],
    ["Constant t-statistics", round(t_stat_a_1927,3), round(t_stat_a_1973,3)],
    ["Dividend Yield", round(b_1927,3), round(b_1973,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_1927,3), round(t_stat_b_1973,3)],
    ["Adjusted R-Squares", round(adj_r2_1927, 3), round(adj_r2_1973, 3)],
]

# Print table in Markdown format
print("Table 4. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Payout Ratio")
print(tabulate(table4, headers="firstrow", tablefmt="pretty"))

Table 4. Regression Results of 1927-2024 and 1973-2024 Real Dividend Growth on Payout Ratio
+-----------------------------+-----------+-----------+
|           Metric            | 1927-2024 | 1973-2024 |
+-----------------------------+-----------+-----------+
|          Constant           |   0.009   |  -0.023   |
|    Constant t-statistics    |   0.129   |  -0.176   |
|       Dividend Yield        |   0.031   |   0.129   |
| Dividend Yield t-statistics |   0.243   |   0.439   |
|     Adjusted R-Squares      |  -0.008   |   0.021   |
+-----------------------------+-----------+-----------+


Don't predict because R-square is very small. T-statistics are small and are not significant at 5% significance level. 

# 5

In [24]:
# seemingly unrelated regression

# dividend yield
X = -np.log(q1['DP'][:-1]).reset_index(drop=True)

# dividend growth. Plus 1 because "Real DG" is net growth
Y1 = np.log(q1['Real DG'][1:]+1).reset_index(drop=True)

# dividend yield
Y2 = -np.log(q1['DP'][1:]).reset_index(drop=True)

# drop NA
data = pd.DataFrame({'X': X, 'Y1': Y1, 'Y2': Y2})

# Drop rows with any NA values
data_cleaned = data.dropna()

equations = {
    "eq1": {
        "dependent": data_cleaned["Y1"],
        "exog": sm.add_constant(data_cleaned["X"])
    },
    "eq2": {
        "dependent": data_cleaned["Y2"],
        "exog": sm.add_constant(data_cleaned["X"])
    }
}

# Fit SUR model
sur_model = SUR(equations)
results = sur_model.fit(cov_type='robust')

# Extract covariance matrix of betas
cov_matrix = results.cov

In [25]:
cov_matrix

,eq1_const,eq1_X,eq2_const,eq2_X
eq1_const,0.013248,-0.003882,-0.003733,0.001073
eq1_X,-0.003882,0.001155,0.001073,-0.000310
eq2_const,-0.003733,0.001073,0.017301,-0.004892
eq2_X,0.001073,-0.000310,-0.004892,0.001401


In [26]:
cov_beta = cov_matrix['eq1_X'][3]

/tmp/ipykernel_2700/725268864.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cov_beta = cov_matrix['eq1_X'][3]


In [27]:
# equation (7)
X = sm.add_constant(data_cleaned['X'])

eq7 = sm.OLS(data_cleaned['Y1'], X).fit(cov_type='HC3')

a_eq7 = eq7.params['const']
b_eq7 = eq7.params['X']
t_stat_a_eq7 = eq7.tvalues['const']
t_stat_b_eq7 = eq7.tvalues['X']
adj_r2_eq7 = eq7.rsquared_adj

In [28]:
# equation 8
# 1927-2024
eq8 = sm.OLS(data_cleaned['Y2'], X).fit(cov_type='HC3')

a_eq8 = eq8.params['const']
b_eq8 = eq8.params['X']
t_stat_a_eq8 = eq8.tvalues['const']
t_stat_b_eq8 = eq8.tvalues['X']
adj_r2_eq8 = eq8.rsquared_adj

In [29]:
print(a_eq7)
print(b_eq7)
print(a_eq8)
print(b_eq8)

0.00033943825295333423
0.004569922590988625
0.1633864956989673
0.9555690176402327


\begin{align*}
Var(\beta_{pd,r}^{Null})&=Var(-(1-\hat{\beta}_{pd,\Delta d}-\rho\hat{\beta}_{pd,pd+1}))\\
&=Var(1-\hat{\beta}_{pd,\Delta d}-\rho\hat{\beta}_{pd,pd+1})\\
&=Var(1)+Var(\beta_{pd,\Delta d})+\rho^2 Var(\beta_{pd,pd+1})+2\rho Cov(\beta_{pd,\Delta d},\beta_{pd,pd+1})
\end{align*}

In [30]:
b_se_eq7 = eq7.bse['X']
b_se_eq8 = eq8.bse['X']

In [31]:
b_var_eq6 = b_se_eq7 ** 2 + (0.94 * b_se_eq8) ** 2 + 2 * 0.94 * cov_beta

In [32]:
b_eq6 = -(1-b_eq7-0.94*b_eq8)
t_stat_b_eq6 = b_eq6 / (b_var_eq6 ** 0.5)

In [33]:
diff_eq6 = np.log(q1['Real R'][1:]).reset_index(drop=True) - b_eq6 * -np.log(q1['DP'][:-1]).reset_index(drop=True)
a_eq6 = diff_eq6.mean()

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [34]:
y = np.log(q1['Real R'][1:]).reset_index(drop=True)

pred = a_eq6 + b_eq6 * -np.log(q1['DP'][:-1]).reset_index(drop=True)
squared_diff = (y - pred) ** 2
sse = squared_diff.sum()

mean = y.mean()

squared_total = (y - mean) ** 2
sst = squared_total.sum()

r2 = 1 - sse/sst

#calculate adjusted r2
n = len(y)
adj_r2_eq6 = 1 - ((1 - r2) * (n - 1)) / (n - 1 - 1)

In [35]:
table5 = [
    ["Metric", "Return", "Dividend Growth", "Dividend Yield"],
    ["Constant", round(a_eq6,3), round(a_eq7,3), round(a_eq8,3)],
    ["Constant t-statistics", "", round(t_stat_a_eq7,3), round(t_stat_a_eq8,3)],
    ["Dividend Yield", round(b_eq6,3), round(b_eq7,3), round(b_eq8,3)],
    ["Dividend Yield t-statistics", round(t_stat_b_eq6,3), round(t_stat_b_eq7,3), round(t_stat_b_eq8,3)],
    ["Adjusted R-Squares", round(adj_r2_eq6,3), round(adj_r2_eq7,3), round(adj_r2_eq8,3)]
]

# Print table in Markdown format
print("Table 5. Regression Results of Log Real Return, Real Dividend Growth, and Real P/D on Log Real P/D")
print(tabulate(table5, headers="firstrow", tablefmt="pretty"))

Table 5. Regression Results of Log Real Return, Real Dividend Growth, and Real P/D on Log Real P/D
+-----------------------------+--------+-----------------+----------------+
|           Metric            | Return | Dividend Growth | Dividend Yield |
+-----------------------------+--------+-----------------+----------------+
|          Constant           | -1.498 |       0.0       |     0.163      |
|    Constant t-statistics    |        |      0.003      |     1.198      |
|       Dividend Yield        | -0.097 |      0.005      |     0.956      |
| Dividend Yield t-statistics | -2.177 |      0.13       |     24.596     |
|     Adjusted R-Squares      | -0.005 |      -0.01      |     0.891      |
+-----------------------------+--------+-----------------+----------------+


# 6

In [ ]:
data = pd.DataFrame({'rt': np.log(q1['Real R']), 'dt': np.log(q1['Real DG']), 'pt-dt': -np.log(q1['DP'])}).dropna()

# Fit the VAR model
model = VAR(data)
var_results = model.fit(maxlags=1)  # Fit the model with lag=1

# Extract the coefficients and residual covariance matrix
A = var_results.coefs  # Transition matrix Γ
A = np.squeeze(A) # change shape from (1,3,3) to (3,3)
epsilon = var_results.sigma_u

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


AttributeError: 'VARResults' object has no attribute 'get_robustcov_results'

In [ ]:
# variance decomposition

rho = 0.94

eI = np.array([1, 0, 0]) # row vector

#identity Matrix
dim = A.shape[1]
I = np.eye(dim)

# (I - rho * A) ^ -1
x = I - rho * A
inv = np.linalg.inv(x)

# news about future returns
news_r = eI @ (rho * A) @ inv @ epsilon

In [ ]:
# news about future cash flows
news_cf = eI @ (I + rho * A @ inv) @ epsilon

In [ ]:
# calculate variance of news and variance of stock market return. 
# Normalise variance of news by variance of stock market return so it sums to 1 
# and variance of news is a percentage of variance of stock market return. Same as Campbell